1\. **Pandas DataFrame**

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every `x` BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If haven't downloaded it yet, please get the data file with wget
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

--2022-11-11 14:18:49--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.69.18, 2620:100:6025:18::a27d:4512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.69.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/xvjzaxzz3ysphme/data_000637.txt [following]
--2022-11-11 14:18:49--  https://www.dropbox.com/s/raw/xvjzaxzz3ysphme/data_000637.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc85bd9542cdb76578580fe646f8.dl.dropboxusercontent.com/cd/0/inline/BwjbPluBdND3nNy1wl-lKPpyERLaalY2MkLfpAyJzNfyO1QY1jIWz5lMCg7nJYcpK-YOuzzJEn3_ssquUD5G64za9qk8_GvzDxBlwgAfcwk30-Y4qAQBK4jo9qFWyWLcB6co3xT1UBi_jrkgIz0539DHBger-JQw_oJQDLKj2v5b_g/file# [following]
--2022-11-11 14:18:50--  https://uc85bd9542cdb76578580fe646f8.dl.dropboxusercontent.com/cd/0/inline/BwjbPluBdND3nNy1wl-lKPpyERLaalY2MkLfpAyJzNfyO1QY1jIWz5lMCg7nJYcpK-YO

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [2]:
import pandas as pd
filename= "./data/data_000637.txt" ##setting the filename
n=10000 
dataset= pd.read_csv(filename,nrows=n) ##pandas function to read a .csv file
print(dataset)


      HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
0        1     0          123  3869200167        2374        26
1        1     0          124  3869200167        2374        27
2        1     0           63  3869200167        2553        28
3        1     0           64  3869200167        2558        19
4        1     0           64  3869200167        2760        25
...    ...   ...          ...         ...         ...       ...
9995     1     0           61  3869200267         696        11
9996     1     0           60  3869200267         701         5
9997     1     0           59  3869200267         707        23
9998     1     0           63  3869200267         706        15
9999     1     0           49  3869200267         777        13

[10000 rows x 6 columns]


2\. Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [3]:
x=dataset["BX_COUNTER"].max() ## accessing to the column and then applying the max() function
print("The max of BX_COUNTER is:", x)

The max of BX_COUNTER is: 3563


3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

In [7]:
absolute_times=[] ##empty list
for i,row in dataset.iterrows():
    time= (row["TDC_MEAS"]*25/30+ row["BX_COUNTER"]*25+ row["ORBIT_CNT"]*x*25) ##applying the definitions
    absolute_times.append(time) ##adding the element to the list
dataset["ABS_TIMES"]=absolute_times ## new column in the dataset
print("Dataset with new column:\n",dataset)
##TODO
import datetime
time_series=pd.to_datetime(dataset["ABS_TIMES"], unit="ns")
print("The time series are:\n",time_series)

Dataset with new column:
       HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS     ABS_TIMES
0        1     0          123  3869200167        2374        26  3.446490e+14
1        1     0          124  3869200167        2374        27  3.446490e+14
2        1     0           63  3869200167        2553        28  3.446490e+14
3        1     0           64  3869200167        2558        19  3.446490e+14
4        1     0           64  3869200167        2760        25  3.446490e+14
...    ...   ...          ...         ...         ...       ...           ...
9995     1     0           61  3869200267         696        11  3.446490e+14
9996     1     0           60  3869200267         701         5  3.446490e+14
9997     1     0           59  3869200267         707        23  3.446490e+14
9998     1     0           63  3869200267         706        15  3.446490e+14
9999     1     0           49  3869200267         777        13  3.446490e+14

[10000 rows x 7 columns]
The time ser

4\. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

In [5]:
dataset= pd.read_csv(filename)

5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [6]:
count=dataset.groupby("TDC_CHANNEL").count() ##counting the iterations of TDC channels
n=3  
noisy=count.sort_values(by="HEAD",ascending=False).iloc[0:n] ##selecting how many channels to show
print(noisy)

               HEAD    FPGA  ORBIT_CNT  BX_COUNTER  TDC_MEAS
TDC_CHANNEL                                                 
139          108059  108059     108059      108059    108059
64            66020   66020      66020       66020     66020
63            64642   64642      64642       64642     64642


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

In [7]:
print("The non empty orbits are:",len(dataset.groupby("ORBIT_CNT").count()))

The non empty orbits are: 11001


7\. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [8]:
print("The orbits with at least one measurament with TDC_CHANNEL=139 are:",len(dataset[dataset["TDC_CHANNEL"]==139].groupby("ORBIT_CNT").count()))

The orbits with at least one measurament with TDC_CHANNEL=139 are: 10976


8\. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

9\. **Optional:** Create two histograms (one for each FPGA) that show the number of counts for each TDC channel.